Packages importieren

In [ ]:
import pandas as pd
import numpy as np
from numpy.linalg import svd
from scipy.optimize import least_squares
import plotly.graph_objects as go
import itertools

Funktionen importieren

In [ ]:
def kabsch_algorithm(P, Q):
    p_centroid = np.mean(P, axis=0)
    q_centroid = np.mean(Q, axis=0)
    P_centered = P - p_centroid # Werte werden ins Zentrum des Koordinatensystems verschoben
    Q_centered = Q - q_centroid # Werte werden ins Zentrum des Koordinatensystems verschoben
    H = P_centered.T @ Q_centered # Korrelationsmatrix
    U, S, Vt = svd(H)
    V = Vt.T
    R = V @ U.T
    if np.linalg.det(R) < 0:
        V[:,-1] *= -1
        R = V @ U.T
    t = q_centroid - R @ p_centroid
    return R, t

def rotation_matrix_from_euler(alpha, beta, gamma):
    # Rotationsmatrizen für Eulerwinkel (Z-Y-X)
    Rz = np.array([[np.cos(alpha), -np.sin(alpha), 0],
                   [np.sin(alpha),  np.cos(alpha), 0],
                   [0,              0,             1]])
    Ry = np.array([[ np.cos(beta), 0, np.sin(beta)],
                   [0,             1, 0],
                   [-np.sin(beta), 0, np.cos(beta)]])
    Rx = np.array([[1,             0,              0],
                   [0, np.cos(gamma), -np.sin(gamma)],
                   [0, np.sin(gamma),  np.cos(gamma)]])
    return Rz @ Ry @ Rx

def residual(params, P, Q):
    alpha, beta, gamma, tx, ty, tz = params
    R_opt = rotation_matrix_from_euler(alpha, beta, gamma)
    t_opt = np.array([tx, ty, tz])
    P_transformed = (R_opt @ P.T).T + t_opt
    diff = P_transformed - Q
    return diff.flatten()

# Introduce noise into 10 random data points
def add_noise_to_random_points(data, noise_level=0.01, num_points=10):
    indices = np.random.choice(data.shape[0], num_points, replace=False)
    noise = np.random.normal(0, noise_level, (num_points, data.shape[1]))
    data_noisy = data.copy()
    data_noisy[indices] += noise
    return data_noisy

def angular_difference(angle1, angle2):
    """Berechnet minimalen Winkelunterschied in Grad."""
    diff = np.abs(angle1 - angle2) % 360
    return np.minimum(diff, 360 - diff)

Messdaten aller Messtage 21.01., 01.02., 10.02., 03.03. (Position 1) und 03.03. (Position 2) importieren

In [51]:
# Daten vom 21.01.2025 importieren und strukturieren
data_ts_21_01 = pd.read_csv('../data/Rohdaten_TS_21_01_25.csv', delimiter=';', index_col=0).sort_index()
data_encoder_21_01 = pd.read_csv('../data/Encoderwerte_21_01_25.csv', delimiter=';', index_col=0).sort_index()
validation_data_TS_21_01 = pd.read_csv('../data/Rohdaten_TS_21_01_V_25.csv', delimiter=';', index_col=0).sort_index()
validation_data_encoder_21_01 = pd.read_csv('../data/Encoderwerte_21_01_V_25.csv', delimiter=';', index_col=0).sort_index()

encoder_points_21_01 = (data_encoder_21_01[['x_Encoder', 'y_Encoder', 'z_Encoder']]/1000)
ts_points_21_01 = data_ts_21_01[['x_TS', 'y_TS', 'z_TS']]

encoder_points_validation_21_01 = validation_data_encoder_21_01[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000
ts_points_validation_21_01 = validation_data_TS_21_01[['x_TS', 'y_TS', 'z_TS']].values 

# Daten vom 01.02.2025 importieren und strukturieren
data = pd.read_csv('../data/Modellentwicklung_Encoderwerte.csv', sep=';', index_col=0)
raw_data = pd.read_csv('../data/Modellentwicklung_Rohdaten_Totalstation.csv', delimiter=';', index_col=0)

raw_data.columns = ['x_TS', 'y_TS', 'z_TS']
data_positioning = data[
    (data['Norm'] < 20) & 
    (data['x_Encoder'] < 2000) & 
    (data['x_Encoder'] > -1200) & 
    (data['y_Encoder'] < 8000) & 
    (data['z_Encoder'] > -1000) & 
    (data['z_Encoder'] < 2200)
]
data_positioning.index = data_positioning.index
raw_data_positioning = raw_data[raw_data.index.isin(data_positioning.index)]
ts_points_01_02 = raw_data_positioning[['x_TS','y_TS','z_TS']].sort_index()
encoder_points_01_02 = data_positioning[['x_Encoder','y_Encoder','z_Encoder']].sort_index()/1000
data_encoder_01_02 = data_positioning[['x_Encoder', 'y_Encoder', 'z_Encoder','encoder_0','encoder_1','encoder_2','encoder_3','encoder_4','encoder_5','encoder_6','encoder_7']].sort_index()
used_indices = set(data_positioning.index)
available_indices = list(set(data.index) - used_indices)
selected_indices = np.random.choice(available_indices, 20, replace=False)
validation_data_encoder_01_02 = data.loc[selected_indices].sort_index()
validation_data_TS_01_02 = raw_data.loc[selected_indices].sort_index()
encoder_points_validation_01_02 = validation_data_encoder_01_02[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000 
ts_points_validation_01_02 = validation_data_TS_01_02[['x_TS', 'y_TS', 'z_TS']].values

# Daten vom 10.02.2025 importieren und strukturieren
data = pd.read_csv('../data/Encoderwerte_Rohdaten_10_02_25.csv', delimiter=';',index_col=0)
ts_points_10_02 = data[['x_TS', 'y_TS', 'z_TS']].sort_index()
encoder_points_10_02 = data[['x_Encoder', 'y_Encoder','z_Encoder']].sort_index()
data_encoder_10_02 = data[['x_Encoder', 'y_Encoder','z_Encoder','encoder_0','encoder_1','encoder_2','encoder_3','encoder_4','encoder_5','encoder_6','encoder_7']].sort_index()

# Daten vom 25.02.2025 importieren und strukturieren
data_TS_25_02 = pd.read_csv('../data/Rohdaten_TS_25_02_E_25.csv', delimiter=';', index_col=0).sort_index()
data_encoder_25_02 = pd.read_csv('../data/Encoderwerte_25_02_E_25.csv', delimiter=';', index_col=0).sort_index()
ts_points_25_02 = data_TS_25_02[['x_TS', 'y_TS', 'z_TS']]
encoder_points_25_02 = data_encoder_25_02[['x_Encoder', 'y_Encoder','z_Encoder']]/1000
validation_data_encoder_25_02 = pd.read_csv('../data/Encoderwerte_25_02_25.csv', delimiter=';', index_col=0).sort_index()
validation_data_TS_25_02 = pd.read_csv('../data/Rohdaten_TS_25_02_25.csv', delimiter=';', index_col=0).sort_index()
encoder_points_validation_25_02 = validation_data_encoder_25_02[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000
ts_points_validation_25_02 = validation_data_TS_25_02[['x_TS', 'y_TS', 'z_TS']].values

# Daten vom 03.03.2025 (Position 1) importieren und strukturieren
data_TS_03_03 = pd.read_csv('../data/Rohdaten_TS_03_03_25_Position_1_Einmessung.csv', delimiter=';', index_col=0).sort_index()
data_encoder_03_03 = pd.read_csv('../data/Encoderwerte_03_03_25_Position_1_Einmessung.csv', delimiter=';', index_col=0).sort_index()
ts_points_03_03 = data_TS_03_03[['x_TS', 'y_TS', 'z_TS']]
encoder_points_03_03 = data_encoder_03_03[['x_Encoder', 'y_Encoder','z_Encoder']]/1000
validation_data_encoder_03_03 = pd.read_csv('../data/Encoderwerte_03_03_25_Position_1.csv', delimiter=';', index_col=0).sort_index()
validation_data_TS_03_03 = pd.read_csv('../data/Rohdaten_TS_03_03_25_Position_1.csv', delimiter=';', index_col=0).sort_index()
validation_data_encoder_03_03 = validation_data_encoder_03_03[~validation_data_encoder_03_03.index.str.endswith(("second_stage_corrected", "first_stage_corrected"))].sort_index()
encoder_points_validation_03_03 = validation_data_encoder_03_03[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000
ts_points_validation_03_03 = validation_data_TS_03_03[['x_TS', 'y_TS', 'z_TS']].values

# Daten vom 03.03.2025 (Position 2) importieren und strukturieren
data_TS_03_03_2 = pd.read_csv('../data/Rohdaten_TS_03_03_25_Position_2_Einmessung.csv', delimiter=';', index_col=0).sort_index()
data_encoder_03_03_2 = pd.read_csv('../data/Encoderwerte_03_03_25_Position_2_Einmessung.csv', delimiter=';', index_col=0).sort_index()
ts_points_03_03_2 = data_TS_03_03_2[['x_TS', 'y_TS', 'z_TS']]
encoder_points_03_03_2 = data_encoder_03_03_2[['x_Encoder', 'y_Encoder','z_Encoder']]/1000

validation_data_encoder_03_03_2 = pd.read_csv('../data/Encoderwerte_03_03_25_Position_2.csv', delimiter=';', index_col=0).sort_index()
validation_data_TS_03_03_2 = pd.read_csv('../data/Rohdaten_TS_03_03_25_Position_2.csv', delimiter=';', index_col=0).sort_index()
validation_data_encoder_03_03_2 = validation_data_encoder_03_03_2[~validation_data_encoder_03_03_2.index.str.endswith(("second_stage_corrected", "first_stage_corrected"))].sort_index()
encoder_points_validation_03_03_2 = validation_data_encoder_03_03_2[['x_Encoder', 'y_Encoder', 'z_Encoder']].values/1000
ts_points_validation_03_03_2 = validation_data_TS_03_03_2[['x_TS', 'y_TS', 'z_TS']].values

# Daten zusammenfassen
datasets = {
    '21_01': (encoder_points_21_01.values, ts_points_21_01.values),
    '01_02': (encoder_points_01_02.values, ts_points_01_02.values),
    '10_02': (encoder_points_10_02.values, ts_points_10_02.values), 
    '25_02': (encoder_points_25_02.values, ts_points_25_02.values),
    '03_03_1': (encoder_points_03_03.values, ts_points_03_03.values),
    '03_03_2': (encoder_points_03_03_2.values, ts_points_03_03_2.values),
}

Transformationen und 3D-Vektorplot für jedes Datenset bestimmen

In [52]:
# Farben für den Vektorplot definieren
colors = {
    '21_01': 'red',
    '01_02': 'blue',
    '10_02': 'green',
    '25_02': 'orange',
    '03_03_1': 'purple',
    '03_03_2': 'cyan'
}

# 3D Vektorplot erstellen
fig = go.Figure()

# Variablen für den for-Loop initialisieren
transformations = {}
counter = 0
x_space = 0
deltas_total = pd.DataFrame()

# For-Loop für die Transformation und den Vektorplot
for day, (P, Q) in datasets.items():
    # Schritt 1: Kabsch
    R_est, t_est = kabsch_algorithm(P, Q)
    
    # Schritt 2: Least Squares Optimierung
    x0 = np.array([0.0, 0.0, 0.0, t_est[0], t_est[1], t_est[2]])
    result = least_squares(residual, x0, args=(P, Q), loss='huber', f_scale=1.0)
    alpha_opt, beta_opt, gamma_opt, tx_opt, ty_opt, tz_opt = result.x
    R_opt = rotation_matrix_from_euler(alpha_opt, beta_opt, gamma_opt)
    t_opt = np.array([tx_opt, ty_opt, tz_opt])

    # Schritt 3: Transformation der Encoderpunkte
    P_transformed = (R_opt @ P.T).T + t_opt

    # Schritt 4: Fehler (Vektoren)
    deltas = (Q - P_transformed) * 1000  # Fehler in mm

    # Lage des Farbbars anpassen
    if counter > 2:
        x_space = 0.1
        counter = 0

    # Schritt 5: Vektorplot erstellen
    quiver = go.Cone(
        x=P[:, 0]*1000,
        y=P[:, 1]*1000,
        z=P[:, 2]*1000,
        u=deltas[:, 0],
        v=deltas[:, 1],
        w=deltas[:, 2],
        opacity=0.5,
        sizemode="raw",
        sizeref=10,  # Anpassen für bessere Sichtbarkeit
        colorscale=[[0, colors[day]], [1, colors[day]]],
        colorbar=dict(
            title=f'Abweichung<br>{day} (mm)',
            len=0.33,
            x=0.85 + x_space,  # Verschieben nach rechts
            y=0.9 - counter*0.33,
        ),
        showscale=True,
        name=f'{day}'
    )

    fig.add_trace(quiver)

    # Rotationsmatrix, Translation, Eulerwinkel und Deltas in Variablen speichern 
    transformations[day] = {'R': R_opt, 't': t_opt, 'Euler': np.degrees([alpha_opt, beta_opt, gamma_opt])}
    if day != '10_02':
        deltas_total = pd.concat([deltas_total, pd.DataFrame(deltas, columns=['x', 'y', 'z'])], axis=0)
    
    counter += 1

# Schritt 6: Vektorplot-Layout anpassen
fig.update_layout(
    title='3D Vektorplot aller Einmessungen',
    scene=dict(
        xaxis_title="<i>x</i> in mm",
        yaxis_title="<i>y</i> in mm",
        zaxis_title="<i>z</i> in mm"
    )
)

# Schritt 7: Vektorplot speichern
fig.write_html("../results/Diagramme/Einmessverfahren/3D_Vektorplot_alle_Einmessungen.html")

Berechnung der Transformationsunterschiede der unterschiedlichen Messtage

In [ ]:
# Nur relevante Tage verwenden
relevant_days = [day for day in transformations.keys() if day != '03_03_2']

# Alle Kombinationen aus zwei verschiedenen relevanten Tagen bilden
comparison_pairs = list(itertools.combinations(relevant_days, 2))

# Ergebnisse speichern
results = []

for day1, day2 in comparison_pairs:
    euler1 = transformations[day1]['Euler']
    euler2 = transformations[day2]['Euler']
    t1 = transformations[day1]['t']
    t2 = transformations[day2]['t']

    # Richtiger Rotationsunterschied:
    diff_roll = angular_difference(euler1[0], euler2[0])
    diff_pitch = angular_difference(euler1[1], euler2[1])
    diff_yaw = angular_difference(euler1[2], euler2[2])

    # Translationsunterschied
    translation_diff = np.linalg.norm((t1 - t2) * 1000)

    results.append({
        'Vergleich': f'{day1} vs {day2}',
        'Roll Unterschied (°)': diff_roll,
        'Pitch Unterschied (°)': diff_pitch,
        'Yaw Unterschied (°)': diff_yaw,
        'Translationsunterschied (mm)': translation_diff
    })

# In DataFrame umwandeln
df_comparison = pd.DataFrame(results)

# Schön ausgeben
print(df_comparison)


          Vergleich  Roll Unterschied (°)  Pitch Unterschied (°)  \
0    21_01 vs 01_02              0.060356               0.004998   
1    21_01 vs 10_02              0.077657               0.136672   
2    21_01 vs 25_02              0.056914               0.301892   
3  21_01 vs 03_03_1              0.056955               0.386203   
4    01_02 vs 10_02              0.017301               0.141670   
5    01_02 vs 25_02              0.003442               0.306890   
6  01_02 vs 03_03_1              0.003401               0.391201   
7    10_02 vs 25_02              0.020744               0.165220   
8  10_02 vs 03_03_1              0.020702               0.249531   
9  25_02 vs 03_03_1              0.000041               0.084311   

   Yaw Unterschied (°)  Translationsunterschied (mm)  
0             0.109252                     16.200062  
1             0.018782                      5.644995  
2             0.115442                     14.087826  
3             0.135720         